In [59]:
import polars as pl
import re
import os
import dotenv
dotenv.load_dotenv(".env")

True

In [60]:
# dataset link https://www.kaggle.com/datasets/hetbabariya/imdb-movies-data-collection-5000-records?resource=download
df = pl.read_csv("IMDB_Movies_Dataset.csv")
df = df.drop("")
df.head()

Title,Average Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime
str,f64,str,str,f64,str,str,str,str,str,str,str
"""The Shawshank Redemption""",9.3,"""Frank Darabont""","""Stephen King, Frank Darabont""",82.0,"""Tim Robbins, Morgan Freeman, B…","""October 14, 1994 (India)""","""United States""","""English""","""$25,000,000 (estimated)""","""$29,332,133""","""2 hours 22 minutes"""
"""Attack on Titan the Movie: The…",9.3,"""Yûichirô Hayashi""","""Natsuki Hanae, Yoshimasa Hosoy…",null,"""Natsuki Hanae, Yoshimasa Hosoy…","""November 8, 2024 (Japan)""","""Japan""","""Japanese""",null,"""$3,513,659""","""2 hours 25 minutes"""
"""The Godfather""",9.2,"""Francis Ford Coppola""","""Mario Puzo, Francis Ford Coppo…",100.0,"""Marlon Brando, Al Pacino, Jame…","""1978 (India)""","""United States""","""English, Italian, Latin""","""$6,000,000 (estimated)""","""$250,342,198""","""2 hours 55 minutes"""
"""Hababam Sinifi""",9.2,"""Ertem Egilmez""","""Umur Bugay, Rifat Ilgaz""",null,"""Münir Özkul, Tarik Akan, Halit…","""April 1, 1975 (Turkey)""","""Turkey""","""Turkish""",null,null,"""1 hour 25 minutes"""
"""Ramayana: The Legend of Prince…",9.2,"""Ram Mohan, Yûgô Sakô, Koichi S…","""Rani Burra, Ram Mohan, Yûgô Sa…",null,"""Arun Govil, Nikhil Kapoor, Edi…","""September 25, 2001 (India)""","""India, Japan""","""English""",null,null,"""2 hours 15 minutes"""


In [61]:
df = df.rename(
    {"Average Rating": "Rating"}
)
df.columns

['Title',
 'Rating',
 'Director',
 'Writer',
 'Metascore',
 'Cast',
 'Release Date',
 'Country of Origin',
 'Languages',
 'Budget',
 'Worldwide Gross',
 'Runtime']

In [62]:
df.schema

Schema([('Title', String),
        ('Rating', Float64),
        ('Director', String),
        ('Writer', String),
        ('Metascore', Float64),
        ('Cast', String),
        ('Release Date', String),
        ('Country of Origin', String),
        ('Languages', String),
        ('Budget', String),
        ('Worldwide Gross', String),
        ('Runtime', String)])

In [63]:
df.null_count()

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,1,1934,7,0,3,21,2338,1094,0


In [64]:
df.glimpse()

Rows: 4989
Columns: 12
$ Title             <str> 'The Shawshank Redemption', 'Attack on Titan the Movie: The Last Attack', 'The Godfather', 'Hababam Sinifi', 'Ramayana: The Legend of Prince Rama', 'The Silence of Swastika', 'Maya Bazaar', 'The Dark Knight', "Schindler's List", '12 Angry Men'
$ Rating            <f64> 9.3, 9.3, 9.2, 9.2, 9.2, 9.1, 9.1, 9.0, 9.0, 9.0
$ Director          <str> 'Frank Darabont', 'Yûichirô Hayashi', 'Francis Ford Coppola', 'Ertem Egilmez', 'Ram Mohan, Yûgô Sakô, Koichi Saski', 'Anuj Bhardwaj', 'Kadiri Venkata Reddy', 'Christopher Nolan', 'Steven Spielberg', 'Sidney Lumet'
$ Writer            <str> 'Stephen King, Frank Darabont', 'Natsuki Hanae, Yoshimasa Hosoya, Marina Inoue', 'Mario Puzo, Francis Ford Coppola', 'Umur Bugay, Rifat Ilgaz', 'Rani Burra, Ram Mohan, Yûgô Sakô', 'Suzanne Bernert', 'Pingali Nagendra Rao, Kadiri Venkata Reddy', 'Jonathan Nolan, Christopher Nolan, David S. Goyer', 'Thomas Keneally, Steven Zaillian', 'Reginald Rose'
$ Metascore     

In [65]:
df.describe()

statistic,Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime
str,str,f64,str,str,f64,str,str,str,str,str,str,str
"""count""","""4989""",4989.0,"""4989""","""4988""",3055.0,"""4982""","""4989""","""4986""","""4968""","""2651""","""3895""","""4989"""
"""null_count""","""0""",0.0,"""0""","""1""",1934.0,"""7""","""0""","""3""","""21""","""2338""","""1094""","""0"""
"""mean""",null,7.632892,null,null,74.684452,null,null,null,null,null,null,null
"""std""",null,0.365243,null,null,12.298911,null,null,null,null,null,null,null
"""min""","""#Home""",7.2,"""A.L. Vijay""","""A.A. Milne, Ernest Shepard, Al…",25.0,"""'Freeway' Ricky Ross, Susan Sa…","""1928 (Italy)""","""Afghanistan, Ireland, Japan, N…","""Aboriginal, Japanese, Hokkien""","""$1,000,000 (estimated)""","""$1,003""","""1 hour"""
"""25%""",null,7.3,null,null,67.0,null,null,null,null,null,null,null
"""50%""",null,7.6,null,null,76.0,null,null,null,null,null,null,null
"""75%""",null,7.8,null,null,84.0,null,null,null,null,null,null,null
"""max""","""色，戒""",9.3,"""Özer Kiziltan""","""Ümit Ünal""",100.0,"""Éric Cantona, Alex Ferguson, M…","""September 9, 2020 (India)""","""Yugoslavia, United States""","""Zulu, Sotho, Xhosa, Afrikaans,…","""₹991,000 (estimated)""","""₹750,000,000""","""9 hours 26 minutes"""


In [66]:
df["Runtime"].value_counts(sort=True)

Runtime,count
str,u32
"""1 hour 35 minutes""",109
"""1 hour 50 minutes""",109
"""1 hour 43 minutes""",100
"""1 hour 58 minutes""",94
"""1 hour 40 minutes""",92
…,…
"""59 minutes""",1
"""7 hours 1 minute""",1
"""5 hours 50 minutes""",1


In [67]:
def runtime_to_minutes(string: str) -> int:
    hours = re.search(r"(\d+) hours?", string)
    if hours:
        hours = int(hours.group(1))
    else:
        hours = 0
    minutes = re.search(r"(\d+) minutes", string)
    if minutes:
        minutes = int(minutes.group(1))
    else:
        minutes = 0
    return hours * 60 + minutes


runtime_to_minutes("2 hours 25 minutes")

145

In [68]:
df = df.with_columns(
    pl.col("Runtime")
    .map_elements(runtime_to_minutes, return_dtype=int)
    .alias("Runtime (minutes)")
)
df.head()

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes)
str,f64,str,str,f64,str,str,str,str,str,str,str,i64
"""The Shawshank Redemption""",9.3,"""Frank Darabont""","""Stephen King, Frank Darabont""",82.0,"""Tim Robbins, Morgan Freeman, B…","""October 14, 1994 (India)""","""United States""","""English""","""$25,000,000 (estimated)""","""$29,332,133""","""2 hours 22 minutes""",142
"""Attack on Titan the Movie: The…",9.3,"""Yûichirô Hayashi""","""Natsuki Hanae, Yoshimasa Hosoy…",null,"""Natsuki Hanae, Yoshimasa Hosoy…","""November 8, 2024 (Japan)""","""Japan""","""Japanese""",null,"""$3,513,659""","""2 hours 25 minutes""",145
"""The Godfather""",9.2,"""Francis Ford Coppola""","""Mario Puzo, Francis Ford Coppo…",100.0,"""Marlon Brando, Al Pacino, Jame…","""1978 (India)""","""United States""","""English, Italian, Latin""","""$6,000,000 (estimated)""","""$250,342,198""","""2 hours 55 minutes""",175
"""Hababam Sinifi""",9.2,"""Ertem Egilmez""","""Umur Bugay, Rifat Ilgaz""",null,"""Münir Özkul, Tarik Akan, Halit…","""April 1, 1975 (Turkey)""","""Turkey""","""Turkish""",null,null,"""1 hour 25 minutes""",85
"""Ramayana: The Legend of Prince…",9.2,"""Ram Mohan, Yûgô Sakô, Koichi S…","""Rani Burra, Ram Mohan, Yûgô Sa…",null,"""Arun Govil, Nikhil Kapoor, Edi…","""September 25, 2001 (India)""","""India, Japan""","""English""",null,null,"""2 hours 15 minutes""",135


In [69]:
def money_to_usd(string: str | None) -> int:
    if not isinstance(string, str) or not "$" in string:
        return None
    budget = int(re.sub(r"[^\d]", "", string))
    return budget
money_to_usd("$6,000,000 (estimated)")

6000000

In [70]:
df = df.with_columns(
    pl.col("Budget").map_elements(money_to_usd, return_dtype=int).alias("Budget (usd)"),
    pl.col("Worldwide Gross").map_elements(money_to_usd, return_dtype=int).alias("Worldwide Gross (usd)"),
)
df.head()

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd)
str,f64,str,str,f64,str,str,str,str,str,str,str,i64,i64,i64
"""The Shawshank Redemption""",9.3,"""Frank Darabont""","""Stephen King, Frank Darabont""",82.0,"""Tim Robbins, Morgan Freeman, B…","""October 14, 1994 (India)""","""United States""","""English""","""$25,000,000 (estimated)""","""$29,332,133""","""2 hours 22 minutes""",142,25000000,29332133
"""Attack on Titan the Movie: The…",9.3,"""Yûichirô Hayashi""","""Natsuki Hanae, Yoshimasa Hosoy…",null,"""Natsuki Hanae, Yoshimasa Hosoy…","""November 8, 2024 (Japan)""","""Japan""","""Japanese""",null,"""$3,513,659""","""2 hours 25 minutes""",145,null,3513659
"""The Godfather""",9.2,"""Francis Ford Coppola""","""Mario Puzo, Francis Ford Coppo…",100.0,"""Marlon Brando, Al Pacino, Jame…","""1978 (India)""","""United States""","""English, Italian, Latin""","""$6,000,000 (estimated)""","""$250,342,198""","""2 hours 55 minutes""",175,6000000,250342198
"""Hababam Sinifi""",9.2,"""Ertem Egilmez""","""Umur Bugay, Rifat Ilgaz""",null,"""Münir Özkul, Tarik Akan, Halit…","""April 1, 1975 (Turkey)""","""Turkey""","""Turkish""",null,null,"""1 hour 25 minutes""",85,null,null
"""Ramayana: The Legend of Prince…",9.2,"""Ram Mohan, Yûgô Sakô, Koichi S…","""Rani Burra, Ram Mohan, Yûgô Sa…",null,"""Arun Govil, Nikhil Kapoor, Edi…","""September 25, 2001 (India)""","""India, Japan""","""English""",null,null,"""2 hours 15 minutes""",135,null,null


In [71]:
df = df.with_columns(
    pl.col("Worldwide Gross (usd)").sub(pl.col("Budget (usd)")).alias("Profit (usd)")
)
df.head()

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd)
str,f64,str,str,f64,str,str,str,str,str,str,str,i64,i64,i64,i64
"""The Shawshank Redemption""",9.3,"""Frank Darabont""","""Stephen King, Frank Darabont""",82.0,"""Tim Robbins, Morgan Freeman, B…","""October 14, 1994 (India)""","""United States""","""English""","""$25,000,000 (estimated)""","""$29,332,133""","""2 hours 22 minutes""",142,25000000,29332133,4332133
"""Attack on Titan the Movie: The…",9.3,"""Yûichirô Hayashi""","""Natsuki Hanae, Yoshimasa Hosoy…",null,"""Natsuki Hanae, Yoshimasa Hosoy…","""November 8, 2024 (Japan)""","""Japan""","""Japanese""",null,"""$3,513,659""","""2 hours 25 minutes""",145,null,3513659,null
"""The Godfather""",9.2,"""Francis Ford Coppola""","""Mario Puzo, Francis Ford Coppo…",100.0,"""Marlon Brando, Al Pacino, Jame…","""1978 (India)""","""United States""","""English, Italian, Latin""","""$6,000,000 (estimated)""","""$250,342,198""","""2 hours 55 minutes""",175,6000000,250342198,244342198
"""Hababam Sinifi""",9.2,"""Ertem Egilmez""","""Umur Bugay, Rifat Ilgaz""",null,"""Münir Özkul, Tarik Akan, Halit…","""April 1, 1975 (Turkey)""","""Turkey""","""Turkish""",null,null,"""1 hour 25 minutes""",85,null,null,null
"""Ramayana: The Legend of Prince…",9.2,"""Ram Mohan, Yûgô Sakô, Koichi S…","""Rani Burra, Ram Mohan, Yûgô Sa…",null,"""Arun Govil, Nikhil Kapoor, Edi…","""September 25, 2001 (India)""","""India, Japan""","""English""",null,null,"""2 hours 15 minutes""",135,null,null,null


In [75]:
highest_budget = df.sort(by="Budget (usd)", descending=True, nulls_last=True)[:10]
highest_budget.head(3)

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd)
str,f64,str,str,f64,str,str,str,str,str,str,str,i64,i64,i64,i64
"""Avengers: Endgame""",8.4,"""Anthony Russo, Joe Russo""","""Christopher Markus, Stephen Mc…",78.0,"""Robert Downey Jr., Chris Evans…","""April 26, 2019 (India)""","""United States""","""English, Japanese, Xhosa, Germ…","""$356,000,000 (estimated)""","""$2,799,439,100""","""3 hours 1 minute""",180,356000000,2799439100,2443439100
"""The Little Mermaid""",7.2,"""Rob Marshall""","""David Magee, Hans Christian An…",59.0,"""Halle Bailey, Jonah Hauer-King…","""May 26, 2023 (India)""","""United States""","""English""","""$355,100,000 (estimated)""","""$569,626,289""","""2 hours 15 minutes""",135,355100000,569626289,214526289
"""Avatar: The Way of Water""",7.5,"""James Cameron""","""James Cameron, Rick Jaffa, Ama…",67.0,"""Sam Worthington, Zoe Saldana, …","""December 16, 2022 (India)""","""United States""","""English""","""$350,000,000 (estimated)""","""$2,320,250,281""","""3 hours 12 minutes""",192,350000000,2320250281,1970250281


In [91]:
# vega-altair under the hood https://altair-viz.github.io/getting_started/overview.html
highest_budget.plot.scatter(
    x="Runtime (minutes)", y="Budget (usd)", color="Profit (usd)"
).properties(width=400, height=400, title="Irises").configure_range(
    ramp="diverging",
)

alt.Chart(...)

In [92]:
df.filter(pl.col("Cast").str.contains("Johnny Depp"))

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd)
str,f64,str,str,f64,str,str,str,str,str,str,str,i64,i64,i64,i64
"""Pirates of the Caribbean: The …",8.1,"""Gore Verbinski""","""Ted Elliott, Terry Rossio, Stu…",63.0,"""Johnny Depp, Geoffrey Rush, Or…","""August 15, 2003 (India)""","""United States""","""English""","""$140,000,000 (estimated)""","""$654,264,546""","""2 hours 23 minutes""",143,140000000,654264546,514264546
"""Platoon""",8.1,"""Oliver Stone""","""Oliver Stone""",92.0,"""Charlie Sheen, Tom Berenger, W…","""February 13, 1989 (India)""","""United States, United Kingdom""","""English, Vietnamese""","""$6,000,000 (estimated)""","""$138,545,632""","""2 hours""",120,6000000,138545632,132545632
"""Edward Scissorhands""",7.9,"""Tim Burton""","""Tim Burton, Caroline Thompson""",74.0,"""Johnny Depp, Winona Ryder, Dia…","""January 1, 1992 (India)""","""United States""","""English""","""$20,000,000 (estimated)""","""$86,024,005""","""1 hour 45 minutes""",105,20000000,86024005,66024005
"""Ed Wood""",7.8,"""Tim Burton""","""Rudolph Grey, Scott Alexander,…",71.0,"""Johnny Depp, Martin Landau, Sa…","""1994 (India)""","""United States""","""English""","""$18,000,000 (estimated)""","""$5,888,045""","""2 hours 7 minutes""",127,18000000,5888045,-12111955
"""What's Eating Gilbert Grape""",7.7,"""Lasse Hallström""","""Peter Hedges""",73.0,"""Johnny Depp, Leonardo DiCaprio…","""March 4, 1994 (United States)""","""United States""","""English""","""$11,000,000 (estimated)""","""$10,032,765""","""1 hour 58 minutes""",118,11000000,10032765,-967235
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Rango""",7.3,"""Gore Verbinski""","""John Logan, Gore Verbinski, Ja…",75.0,"""Johnny Depp, Isla Fisher, Timo…","""April 20, 2011 (India)""","""United States, United Kingdom""","""English""","""$135,000,000 (estimated)""","""$245,724,603""","""1 hour 47 minutes""",107,135000000,245724603,110724603
"""Lost in La Mancha""",7.3,"""Keith Fulton, Louis Pepe""","""Keith Fulton, Louis Pepe""",75.0,"""Terry Gilliam, Johnny Depp, Je…","""August 2, 2002 (United Kingdom…","""United Kingdom, United States""","""English, Spanish, French""",null,"""$1,407,019""","""1 hour 33 minutes""",93,null,1407019,null
"""Chocolat""",7.2,"""Lasse Hallström""","""Joanne Harris, Robert Nelson J…",64.0,"""Juliette Binoche, Johnny Depp,…","""August 1, 2021 (India)""","""United Kingdom, United States""","""English, French""","""$25,000,000 (estimated)""","""$152,700,626""","""2 hours 1 minute""",120,25000000,152700626,127700626


In [95]:
df.group_by("Director").agg(total_gross=pl.col("Worldwide Gross (usd)").sum()).sort(
    by="total_gross", descending=True
)[:10]

Director,total_gross
str,i64
"""James Cameron""",8705319635
"""Steven Spielberg""",7988558298
"""Anthony Russo, Joe Russo""",6721322058
"""Peter Jackson""",6484962056
"""Christopher Nolan""",6205459936
"""David Yates""",5003353447
"""Jon Watts""",3966372984
"""Joss Whedon""",2966001713
"""J.J. Abrams""",2924357232


In [96]:
df = df.with_columns(
    pl.col("Release Date")
    .map_elements(lambda x: int(re.search(r"\d{4}", x).group()), return_dtype=int)
    .alias("Year")
)
df = df.with_columns(
    pl.col("Year")
    .map_elements(lambda x: round(x, -1), return_dtype=int)
    .alias("Decade")
)
df[["Release Date", "Year", "Decade"]].head()

Release Date,Year,Decade
str,i64,i64
"""October 14, 1994 (India)""",1994,1990
"""November 8, 2024 (Japan)""",2024,2020
"""1978 (India)""",1978,1980
"""April 1, 1975 (Turkey)""",1975,1980
"""September 25, 2001 (India)""",2001,2000


In [97]:
df.write_parquet("polars_parquet", partition_by="Decade")

In [98]:
# https://docs.pola.rs/user-guide/sql/intro/#register-dataframes
# Polars translates SQL queries into expressions, which are then executed using its own engine
query = """
    SELECT * FROM df
    WHERE REGEXP_LIKE(`Cast`, '.*Freeman.*')
"""
pl.sql(query).collect().head()

Title,Rating,Director,Writer,Metascore,Cast,Release Date,Country of Origin,Languages,Budget,Worldwide Gross,Runtime,Runtime (minutes),Budget (usd),Worldwide Gross (usd),Profit (usd),Year,Decade
str,f64,str,str,f64,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64
"""The Shawshank Redemption""",9.3,"""Frank Darabont""","""Stephen King, Frank Darabont""",82.0,"""Tim Robbins, Morgan Freeman, B…","""October 14, 1994 (India)""","""United States""","""English""","""$25,000,000 (estimated)""","""$29,332,133""","""2 hours 22 minutes""",142,25000000,29332133,4332133,1994,1990
"""The Dark Knight""",9.0,"""Christopher Nolan""","""Jonathan Nolan, Christopher No…",84.0,"""Christian Bale, Heath Ledger, …","""July 18, 2008 (India)""","""United States, United Kingdom""","""English, Mandarin""","""$185,000,000 (estimated)""","""$1,009,053,678""","""2 hours 32 minutes""",152,185000000,1009053678,824053678,2008,2010
"""Se7en""",8.6,"""David Fincher""","""Andrew Kevin Walker""",65.0,"""Morgan Freeman, Brad Pitt, Kev…","""September 22, 1995 (India)""","""United States""","""English""","""$33,000,000 (estimated)""","""$327,333,559""","""2 hours 7 minutes""",127,33000000,327333559,294333559,1995,2000
"""Back to the Future""",8.5,"""Robert Zemeckis""","""Robert Zemeckis, Bob Gale""",87.0,"""Michael J. Fox, Christopher Ll…","""December 9, 1986 (India)""","""United States""","""English""","""$19,000,000 (estimated)""","""$384,577,472""","""1 hour 56 minutes""",116,19000000,384577472,365577472,1986,1990
"""The Dark Knight Rises""",8.4,"""Christopher Nolan""","""Jonathan Nolan, Christopher No…",78.0,"""Christian Bale, Tom Hardy, Ann…","""July 20, 2012 (India)""","""United States, United Kingdom""","""English, Arabic""","""$250,000,000 (estimated)""","""$1,114,975,066""","""2 hours 44 minutes""",164,250000000,1114975066,864975066,2012,2010


In [99]:
# requires connectorx
# read_database_uri
pl.read_database_uri(
    "SELECT * FROM orders",
    os.environ["SQL_CONNECTION"]
)

id,created_at,updated_at,uploaded_at,data
i32,"datetime[μs, UTC]","datetime[μs, UTC]","datetime[μs, UTC]",str
1,2025-01-08 11:50:01 UTC,2025-01-08 11:50:01 UTC,2025-01-08 12:00:00 UTC,"""some string"""
2,2025-01-08 11:50:15 UTC,2025-01-08 11:50:15 UTC,2025-01-08 12:00:00 UTC,"""some other string"""
3,2025-01-08 11:51:06 UTC,2025-01-08 11:52:31 UTC,2025-01-08 12:00:00 UTC,"""other string"""
4,2025-01-08 11:52:01 UTC,2025-01-08 12:11:01 UTC,2025-01-08 12:00:00 UTC,"""changed order"""
5,2025-01-08 11:52:15 UTC,2025-01-08 11:52:15 UTC,2025-01-08 12:00:00 UTC,"""another string"""
6,2025-01-08 11:52:26 UTC,2025-01-08 11:54:31 UTC,2025-01-08 12:00:00 UTC,"""yolo string"""


Performance:
https://towardsdatascience.com/high-performance-data-processing-pandas-2-vs-polars-a-vcpu-perspective-e922d3064f4e